In [3]:
from pgsql_search.loader import HuggingFaceDatasets

ds = HuggingFaceDatasets("UCSC-VLAA/Recap-COCO-30K")
ds = ds.select(list(range(10)))
ds.save_images("../data/images10")
ds = ds.select_columns(["image_id", "image_filepath", "caption"])
ds.dataset


2024-12-18 16:25:03.125 | INFO     | pgsql_search.loader:__init__:15 - Loading dataset: UCSC-VLAA/Recap-COCO-30K
2024-12-18 16:25:03.127 | INFO     | pgsql_search.loader:save_images:44 - Saving images to folder: ../data/images10


Dataset({
    features: ['image_id', 'image_filepath', 'caption'],
    num_rows: 10
})

In [4]:
df = ds.dataset.to_pandas()
df

,image_id,image_filepath,caption
0,53120,../data/images10/53120.jpg,Several horses with riders crossing a body of ...
1,325873,../data/images10/325873.jpg,A woman and a child sit on the floor and inter...
2,217937,../data/images10/217937.jpg,There is an open face sandwich and a fork on t...
3,196865,../data/images10/196865.jpg,a pillow a bed a table and two books
4,4229,../data/images10/4229.jpg,Three elephants in a field next to each other.
5,353028,../data/images10/353028.jpg,A plate of cheese pizza on a dinner table.
6,539738,../data/images10/539738.jpg,A man in a tie and shirt holding a suitcase an...
7,155811,../data/images10/155811.jpg,Several people walking on sidewalks with umbre...
8,431058,../data/images10/431058.jpg,A snow boarder in the air near a couple of pin...
9,274678,../data/images10/274678.jpg,a giraffe standing along in the woods by its self


In [5]:
from pgsql_search.database import PostgreSQLDatabase, ColumnType

In [7]:
with PostgreSQLDatabase("retrieval_db") as db:
    # First, create the table with just an ID column
    db.initialize_table("image_metadata")
    db.add_column("image_filepath", ColumnType.TEXT, nullable=False)
    db.add_column("caption", ColumnType.TEXT, nullable=True)


    db.insert_dataframe(df)

2024-12-18 16:26:44.696 | INFO     | pgsql_search.database:connect:87 - Connected to database
2024-12-18 16:26:44.699 | INFO     | pgsql_search.database:setup_pgvector_extension:104 - pgvector extension initialized
2024-12-18 16:26:44.702 | INFO     | pgsql_search.database:initialize_table:126 - Initialized table 'image_metadata' with ID column 'id'
2024-12-18 16:26:44.704 | INFO     | pgsql_search.database:add_columns:175 - Added 1 new columns to image_metadata
2024-12-18 16:26:44.705 | INFO     | pgsql_search.database:add_columns:175 - Added 1 new columns to image_metadata
2024-12-18 16:26:44.709 | ERROR    | pgsql_search.database:insert_dataframe:241 - Error inserting data: null value in column "image_filepaths" of relation "image_metadata" violates not-null constraint
DETAIL:  Failing row contains (1, null, Several horses with riders crossing a body of water.).
2024-12-18 16:26:44.709 | INFO     | pgsql_search.database:disconnect:96 - Disconnected from database
2024-12-18 16:26:44.

NotNullViolation: null value in column "image_filepaths" of relation "image_metadata" violates not-null constraint
DETAIL:  Failing row contains (1, null, Several horses with riders crossing a body of water.).